# Build Final Exam Data

In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '../cmds')
from ratecurves import *
from ficcvol import *
from binomial import *

from datetime import datetime
import warnings

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)

***

In [3]:
DATE = '2023-05-05'
FILEIN = f'../build_data/cap_curves_{DATE}.xlsx'

### Export Curves to Excel

In [4]:
SAVEDATA = False
FILEOUT = f'exam_data_{DATE}'

# Market Data

### Load Curves

In [5]:
Tdata = 3.25
curves = pd.read_excel(FILEIN, sheet_name=f'rate curves {DATE}').set_index('tenor')
curves

,swap rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,
0.25,0.050719,0.987479,NaN,NaN,NaN
0.50,0.050123,0.975402,0.049527,0.316162,0.316162
0.75,0.048560,0.964447,0.045435,0.347778,0.372145
1.00,0.046639,0.954690,0.040881,0.379394,0.428165
1.25,0.044145,0.946600,0.034187,0.461469,0.668067
1.50,0.041930,0.939349,0.030874,0.520051,0.698306
1.75,0.040224,0.932356,0.030003,0.558612,0.688861
2.00,0.038818,0.925648,0.028986,0.580620,0.659912
2.25,0.037466,0.919518,0.026665,0.589547,0.612262


In [6]:
curves = curves.loc[:Tdata,['discounts','fwd vols']]
curves

,discounts,fwd vols
tenor,,
0.25,0.987479,NaN
0.50,0.975402,0.316162
0.75,0.964447,0.372145
1.00,0.954690,0.428165
1.25,0.946600,0.668067
1.50,0.939349,0.698306
1.75,0.932356,0.688861
2.00,0.925648,0.659912
2.25,0.919518,0.612262


### BDT Tree

In [7]:
quotes = curves['discounts']*100

sigmas = curves['fwd vols']
sigmas.iloc[0] = sigmas.iloc[1]

theta, ratetree = estimate_theta(sigmas,quotes)
format_bintree(theta.to_frame().T, style='{:.2%}')

format_bintree(ratetree,style='{:.2%}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,5.04%,5.69%,6.11%,6.49%,6.50%,7.13%,8.56%,10.37%,12.12%,16.07%,21.02%,27.78%,35.66%
1,,4.15%,4.45%,4.73%,4.74%,5.20%,6.24%,7.56%,8.84%,11.71%,15.33%,20.25%,25.99%
2,,,3.07%,3.26%,3.26%,3.58%,4.30%,5.21%,6.09%,8.07%,10.56%,13.96%,17.92%
3,,,,2.12%,2.13%,2.33%,2.80%,3.39%,3.97%,5.26%,6.88%,9.10%,11.68%
4,,,,,1.09%,1.20%,1.44%,1.74%,2.03%,2.70%,3.53%,4.66%,5.99%
5,,,,,,0.60%,0.71%,0.87%,1.01%,1.34%,1.76%,2.32%,2.98%
6,,,,,,,0.36%,0.43%,0.51%,0.67%,0.88%,1.16%,1.50%
7,,,,,,,,0.22%,0.26%,0.35%,0.46%,0.60%,0.77%
8,,,,,,,,,0.14%,0.19%,0.25%,0.33%,0.42%


## CTD Table

In [8]:
tv = ratetree.copy()[1]
mask = ~tv.isna()

NOMDISC = .06
cpns = np.array([.01,.01,.045,.045,.045])
out = pd.DataFrame(100*price_bond(tv[mask], 2, cpns,face=1, accr_frac = 0) / price_bond(NOMDISC, 2, cpns,face=1, accr_frac = 0)).rename(columns={1:'CTD price at expry'}).round(2)

out

,CTD price at expry
state,
0,99.05
1,102.47
2,105.31
3,107.62
4,109.79


### Save Data

In [9]:
if SAVEDATA:
    outfile = f'../data/{FILEOUT}.xlsx'
    with pd.ExcelWriter(outfile) as writer:  
        curves.to_excel(writer, sheet_name= f'rate curves', index=True)
        ratetree.to_excel(writer, sheet_name= f'rate tree', index=True)